In [2]:
from google import genai
from google.genai import types
from google.genai.types import GenerateContentConfig
import os

In [3]:
prompt = """
You are a dish idea generator. Your job is to list dishes that can be created with only the ingredients you are given. If a dish cannot be made only with the ingredients given, do not list it! Do not invent new dishes. You do not need to make use of every ingredient. Output as a JSON array. List required ingredients with each dish. You may assume that the user has water, salt, and black pepper.

Example:
Ingredients:
[
  "Yogurt",
  "Green Apples",
  "Pickles",
  "Donuts",
  "Cheese",
  "Red Apple",
  "Lemon",
  "Cucumber",
  "Lettuce",
  "Eggplant",
  "Bottled Juice",
  "Celery",
  "Limes",
  "Carrots",
  "Oranges",
  "Tomatoes",
  "Green Bell Peppers",
  "Milk"
]

Dishes:
[
  "Yogurt & Fruit Salad" : {
    "ingredients": ["Yogurt", "Green Apples", "Red Apple", "Oranges", "Lemon"],
  },
  "Cucumber & Yogurt Salad" : {
    "ingredients": ["Yogurt", "Cucumber", "Celery", "Lemon"],
  },
  "Simple Garden Salad" : {
    "ingredients": ["Lettuce", "Cucumber", "Tomatoes", "Green Bell Peppers"],
  },
  "Pickle & Cheese Plate": {
    "ingredients": ["Pickles", "Cheese"],
  },
  "Carrot and Celery Sticks with Yogurt Dip": {
    "ingredients": ["Yogurt", "Carrots", "Celery"],
  },
  "Fruit Smoothie": {
    "ingredients": ["Green Apples", "Oranges", "Lemon", "Bottled Juice"],
  },
  "Lemon-Lime Refresher": {
    "ingredients": ["Lemon", "Limes", "Bottled Juice"],
  },
  "Open-Faced Sandwich": {
    "ingredients": ["Cheese", "Tomatoes", "Lettuce"],
  },
  "Grilled Cheese Sandwich": {
    "ingredients": ["Cheese", "Bread"],
  },
  "Cheese and Apple Slices": {
    "ingredients": ["Cheese", "Green Apples"],
  }
]
"""

In [4]:
query = """
Ingredients:
[
  "Yogurt",
  "Green Apples",
  "Pickles",
  "Donuts",
  "Cheese",
  "Red Apple",
  "Lemon",
  "Cucumber",
  "Lettuce",
  "Eggplant",
  "Bottled Juice",
  "Celery",
  "Limes",
  "Carrots",
  "Oranges",
  "Tomatoes",
  "Green Bell Peppers",
  "Milk"
]
"""

In [5]:
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

config = GenerateContentConfig(
    temperature = 0.5,
    max_output_tokens = 2048
)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[prompt, query],
    config=config)

print(response.text)

```json
[
  {
    "Yogurt & Fruit Salad": {
      "ingredients": [
        "Yogurt",
        "Green Apples",
        "Red Apple",
        "Oranges",
        "Lemon"
      ]
    }
  },
  {
    "Cucumber & Yogurt Salad": {
      "ingredients": [
        "Yogurt",
        "Cucumber",
        "Celery",
        "Lemon"
      ]
    }
  },
  {
    "Simple Garden Salad": {
      "ingredients": [
        "Lettuce",
        "Cucumber",
        "Tomatoes",
        "Green Bell Peppers"
      ]
    }
  },
  {
    "Pickle & Cheese Plate": {
      "ingredients": [
        "Pickles",
        "Cheese"
      ]
    }
  },
  {
    "Carrot and Celery Sticks with Yogurt Dip": {
      "ingredients": [
        "Yogurt",
        "Carrots",
        "Celery"
      ]
    }
  },
  {
    "Fruit Smoothie": {
      "ingredients": [
        "Green Apples",
        "Oranges",
        "Lemon",
        "Bottled Juice"
      ]
    }
  },
  {
    "Lemon-Lime Refresher": {
      "ingredients": [
        "Lemon",
        "Li

In [6]:
import json
dishes = json.loads(response.text[response.text.find("["):response.text.rfind("]") + 1])

In [10]:
for dish in dishes:
    print(dish)

{'Yogurt & Fruit Salad': {'ingredients': ['Yogurt', 'Green Apples', 'Red Apple', 'Oranges', 'Lemon']}}
{'Cucumber & Yogurt Salad': {'ingredients': ['Yogurt', 'Cucumber', 'Celery', 'Lemon']}}
{'Simple Garden Salad': {'ingredients': ['Lettuce', 'Cucumber', 'Tomatoes', 'Green Bell Peppers']}}
{'Pickle & Cheese Plate': {'ingredients': ['Pickles', 'Cheese']}}
{'Carrot and Celery Sticks with Yogurt Dip': {'ingredients': ['Yogurt', 'Carrots', 'Celery']}}
{'Fruit Smoothie': {'ingredients': ['Green Apples', 'Oranges', 'Lemon', 'Bottled Juice']}}
{'Lemon-Lime Refresher': {'ingredients': ['Lemon', 'Limes', 'Bottled Juice']}}
{'Cheese and Apple Slices': {'ingredients': ['Cheese', 'Green Apples']}}
{'Apple slices with Yogurt': {'ingredients': ['Yogurt', 'Red Apple']}}
{'Carrot and Celery Juice': {'ingredients': ['Carrots', 'Celery', 'Bottled Juice']}}
{'Orange Juice': {'ingredients': ['Oranges']}}
{'Tomato and Cucumber Salad': {'ingredients': ['Tomatoes', 'Cucumber']}}


In [ ]:
from duckduckgo_search import DDGS
from duckduckgo_search.exceptions import DuckDuckGoSearchException
import requests
import time
from bs4 import BeautifulSoup

for dish in dishes:
    dish_name = list(dish.keys())[0]
    ingredients = dish[dish_name]['ingredients']
    query = dish_name
    print(query)
    while True:
        try:
            results = DDGS().text(query, max_results=20)
            break
        except DuckDuckGoSearchException:
            time.sleep(1)

    for result in results[:1]:
        content = requests.get(result['href']).text
        soup = BeautifulSoup(content, 'html.parser')
        content = soup.get_text()
        if "Ingredients" in content:
            ingredient_list_raw = content[content.find("Ingredients"):content.find("Ingredients") + 1000]
            response = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=["What are the ingredients in this recipe?", ingredient_list_raw],
                config=config)
            print(response.text)

Yogurt & Fruit Salad
Cucumber & Yogurt Salad
The ingredients in this recipe are:

*   2 cucumbers
*   1 cup plain yogurt
*   1 teaspoon dried dill (or a couple teaspoons of fresh dill)
*   Salt
*   Pepper
Simple Garden Salad
Based on the recipe you provided, here are the ingredients:

*   1 medium head Iceberg lettuce
*   1 cup cherry Tomatoes
*   4 slices cooked bacon
*   2 hard boiled eggs
*   1/2 cup croutons
*   1/4 medium red onion (or 1/4 cup chopped chives)
*   1 recipe Buttermilk Ranch Dressing (the ingredients for this dressing are not listed in your provided text)
Pickle & Cheese Plate
Here are the ingredients listed in the recipe:

*   1 cup Fried Pickle Dip
*   8 oz Baked Salami Chips (or plain salami)
*   16 oz potato chips
*   8 strips bacon, broken in half
*   1 jar sweet pickles, drained
*   1 jar dill pickles, drained
*   1 jar bread and butter pickles, drained
*   1 jar dill pickle spears, drained
Carrot and Celery Sticks with Yogurt Dip
The ingredients in this recipe

KeyboardInterrupt: 